# Домашнее задание

На семинаре мы разбирались с классификаторами, а в домашнем задании будем разбираться с моделями регрессии.

Но не бойтесь: принципы работы похожи, а мы оставили ссылки на документацию и небольшие подсказки

Для работы мы выбрали датасет Ames Housing.

Мы подготовили две модели регрессии, с которыми в будете работать и обучать на них ансамблевые модели: линейную регрессию и модель 10-ближайших соседей, чтобы вам не пришлось мучаться с выбором моделей :)

In [1]:
import pandas as pd
df_housing = pd.read_csv("AmesHousing.txt", sep="\t")

X = df_housing[["Gr Liv Area", "Bedroom AbvGr", "Full Bath"]]
y = df_housing["SalePrice"]

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split


(feat_train, feat_test, tar_train, tar_test) = train_test_split(X, y, test_size=0.1, random_state=42)


# Обучим линейную модель
linear_model = LinearRegression()
linear_model.fit(X=feat_train, y=tar_train)

# Обучим модель модель 10 ближайших соседей
knn_model = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor(n_neighbors=10)
)
knn_model.fit(X=feat_train, y=tar_train)

# Получим предсказание для дома с признаками new_house = [[1500, 2, 2]]
new_house = [[1500, 2, 2]]
linear_model.predict(new_house), knn_model.predict(new_house)

/Library/Python/3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Library/Python/3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


(array([216665.27072791]), array([211395.8]))

Модель $k$-ближайших соседей кажется чуть более подходящим вариантом, чем обычная линейная регрессия. Давайте скомбинируем их и попробуем получить новые результаты

### **[Задание 1]**

Начнем с [`VotingRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingRegressor.html#sklearn.ensemble.VotingRegressor).

Импортируйте этот класс, инициализируйте модель с `estimator`, состоящим из наших обученных моделей линейной регрессии и $k$-ближайших соседей.




In [4]:
from sklearn.ensemble import VotingRegressor

# Создаем VotingRegressor
voting_regressor = VotingRegressor(
    estimators=[
        ('linear', linear_model),
        ('knn', knn_model)
    ]
)

### **[Задание 2]**

Запустите обучение на тренировочной выборке.




In [5]:
voting_regressor.fit(X=feat_train, y=tar_train)

VotingRegressor(estimators=[('linear', LinearRegression()),
                            ('knn',
                             Pipeline(steps=[('standardscaler',
                                              StandardScaler()),
                                             ('kneighborsregressor',
                                              KNeighborsRegressor(n_neighbors=10))]))])

### **[Задание 3]**

Постройте предсказание для объекта `new_house` и для тестовой выборки.


In [6]:
predictions_voting = voting_regressor.predict(new_house)
predictions_voting

/Library/Python/3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Library/Python/3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([214030.53536396])

### **[Задание 4]**

Вычислите метрики качества модели. MSE и RMSE будут интересным вариантом, но вы можете выбрать любые две подходящие из этой [страницы документации](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwi7vICTy-qAAxWZHBAIHdudDEMQFnoECBAQAQ&url=http%3A%2F%2Fscikit-learn.org%2Fstable%2Fmodules%2Fmodel_evaluation.html&usg=AOvVaw0dp86tZ28Ezx0G4H5I8rZY&opi=89978449)

In [7]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Получим предсказания модели для тестовых данных
predictions_voting_test = voting_regressor.predict(feat_test)

# Вычислим MSE и RMSE для модели
mse_voting = mean_squared_error(tar_test, predictions_voting_test)
rmse_voting = np.sqrt(mse_voting)


### **[Задание 5]**

C `VotingRegressor` разобрались, а теперь вам необходимо реализовать еще два алгоритма: [`RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) и [`StackingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html#sklearn.ensemble.StackingClassifier).

Выполните задания 1-4 для этих двух алгоритмов.

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Создаем и обучаем RandomForestClassifier
random_forest_classifier = RandomForestClassifier(random_state=42)
random_forest_classifier.fit(X=feat_train, y=tar_train)

# Получим предсказания для тестовых данных
predictions_rf = random_forest_classifier.predict(feat_test)

# Вычислим метрики качества модели
accuracy_rf = accuracy_score(tar_test, predictions_rf)
accuracy_rf

0.006825938566552901

In [10]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Определяем базовые классификаторы
base_classifiers = [
    ('rf', RandomForestClassifier(random_state=42)),
    ('dt', DecisionTreeClassifier(random_state=42))
]

# Создаем и обучаем StackingClassifier
stacking_classifier = StackingClassifier(
    estimators=base_classifiers,
    final_estimator=LogisticRegression()
)
stacking_classifier.fit(X=feat_train, y=tar_train)

# Получим предсказания для тестовых данных
predictions_stacking = stacking_classifier.predict(feat_test)

# Вычислим метрики качества модели
accuracy_stacking = accuracy_score(tar_test, predictions_stacking)
accuracy_stacking


/Library/Python/3.9/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Library/Python/3.9/site-packages/sklearn/model_selection/_validation.py:1163: RuntimeWarning: Number of classes in training fold (840) does not match total number of classes (961). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn(
/Library/Python/3.9/site-packages/sklearn/model_selection/_validation.py:1163: RuntimeWarning: Number of classes in training fold (839) does not match total number of classes (961). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn(
/Library/Python/3.9/site-packages/sklearn/model_selection/_validation.py:1163: RuntimeWarning: Number of classes in training fold (843) does not ma

0.017064846416382253